# 符合Python风格的对象

在Python中，自定义类的行为能够和内置类型一样自然。这一特性得益于鸭子类型(duck type) —— 仅需要按照预定的行为实现对象指定的方法即可。

本章主要是讨论了如何让一个自定义类表现得像一个内置类以及如何扩展格式化输出功能（format()等），并以一个实现各种内置方法的向量类来展示上述话题。

## 基础向量类

现在想实现一个基础的向量类，并且该类具有如下功能：

1. 可以直接访问类实例的属性，而不需要通过调用方法
2. 可以直接拆包
3. 可以被repr()函数正常调用
4. 可以使用==运算符判断相等性
5. 可以被print()函数正常调用
6. 可以被bytes()函数正常调用
7. 可以被abs()函数正常调用并返回向量的模
8. 可以被bool()函数正常调用并当向量的模为0时返回False，否则返回True
9. 从字节序列重建实例

对于上述的功能有如下逐条分析：

1. 默认可行，可以使用cls.param来获取cls类实例中param属性的值
2. 需要实现\_\_iter\_\_()方法
3. 需要实现\_\_repr\_\_()方法
4. 需要实现\_\_eq\_\_()方法
5. 需要实现\_\_str\_\_()方法
6. 需要实现\_\_bytes\_\_()方法
7. 需要实现\_\_abs\_\_()方法
8. 需要实现\_\_bool\_\_()方法
9. 需要实现对应的构造方法

## classmethod & staticmethod

classmethod装饰的方法旨在操作类，这些方法第一个参数传入的是类本身；而staticmethod装饰的方法仅是一个位于类定义体中的静态方法，类似于一个普通的函数。

In [ ]:
from array import array
import math


class BasicVector2d:
    typecode = "d"

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        """
        从字节序列中创建序列
        1. 解析类型字节
        2. 解析剩余字节码
        """
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

## 格式化输出

在上述向量类的基础上添加自定义的格式化输出方法以便于更好的打印上述向量类的实例

目标：
* 支持格式化输出
* 自定义向量的极坐标格式化输出

操作：
* 实现\_\_format\_\_()方法
* 在\_\_format\_\_()中添加自定义的极坐标格式化输出模式

format()函数以及str.format()方法会将各个类型的格式化方法实现委托给\_\_format\_\_()方法

正由于format()函数会主动调用\_\_format\_\_()方法实现格式化，因此具体如何实现格式化输出由各个类决定。\_\_format\_\_()方法可以自行决定如何解析传入的格式说明。

若没有实现\_\_format\_\_()方法，则会自动调用\_\_str\_\_()方法，但是此时不可以传入格式说明，否则会报错。

为了实现格式化输出，需要将传入的格式说明作用于自定义的向量类的每一个分量上，因此有如下的实现：

```Python
def __format__(self, fmt_spec=""):
    components = (format(c, fmt_spec) for c in self)
    return "({}, {})".format(*components)
```

上述示例实现了向量类的格式化输出，能够解析一般格式说明。

为了实现极坐标格式化输出，\_\_format\_\_()方法需要特别支持极坐标格式说明。假设用字符“p”表示要求以极坐标的形式进行格式化输出，\_\_format\_\_()可以有如下实现：
```Python

def angle(self):
    return math.atan2(self.y, self.x)

def __format__(self, fmt_spec=""):
    if fmt_spec.endswith("p"):
        fmt_spec = fmt_spec[:-1]
        coords = (abs(self), self.angle())
        outer_fmt = "<{}, {}>"
    else:
        coords = self
        outer_fmt = "({}, {})"
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)
```

上式实现首先判断格式说明是否以"p"结尾，若以p结尾则启用极坐标格式 —— 首先计算极坐标的模长和弧度，然后使用特定的"<>"表示输出是极坐标；否则直接按照一般形式输出。

这一实现实际上暗含了一个要求：极坐标格式说明符“p”需要置于格式说明的结尾，这一假设在应用中可能引起用户的困扰，因此可以使用"in"关键字判断字符“p”是否存在于fmt_spec，然后对fmt_spec进行解析。


## 可散列的自定义类

若想将自定义类置于集合中则要求该自定义类是可散列的。由于上述的向量类没有实现\_\_hash\_\_()方法，因此此时的向量类是不可散列的。

目标：
* 自定义类实例可散列

操作：
* 实现\_\_hash\_\_()方法

实现\_\_hash\_\_()方法以及\_\_eq\_\_()方法后自定义类的实例就是可散列的，实际上也可以基于\_\_hash\_\_()方法重写\_\_eq\_\_()方法 —— 对象的hash值在整个生命周期中不会发生变化。

按照文档的建议，最好是混合各个属性的散列值构建该对象的hash值。[it is advised to mix together the hash values of the components of the object that also play a part in comparison of objects by packing them into a tuple and hashing the tuple](https://docs.python.org/3/reference/datamodel.html)。

但是，显然，对于一个自定义的向量类的实例来说，其属性值是可以变化的，此时有两种可选的方法：

1. 强制要求属性值不可变，即设定属性值仅只读
2. 在尝试改变属性值时，传回一个新的实例

本书采用的是第一种方法，即实现了自定义向量类属性的只读特性。

第二种方法实际上也很容易实现：在进行赋值时（使用@param.setter装饰器装饰赋值函数）主动创建新实例并返回
```Python

def __init__(self, x, y):
    self.__x = float(x)
    self.__y = float(y)

...

@property
def x(self):
    return self.__x

@property
def y(self):
    return self.__y

def __hash__(self):
    return hash(self.x) ^ hash(self.y)

```

上述实现一方面能兼容之前的功能，此外还让用户无法直接访问属性或是尝试修改属性。

整合上述功能可以有如下的自定义向量类

In [16]:
from array import array
import math


class Vector2d:
    typecode = "d"

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        return self.__x

    @property    
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("p"):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:
            coords = self
            outer_fmt = "({}, {})"
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        """
        从字节序列中创建序列
        1. 解析类型字节
        2. 解析剩余字节码
        """
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [17]:
v1 = Vector2d(3, 4)

# 访问属性
print("v1向量 x分量: ", v1.x)
print("v1向量 y分量: ", v1.y)

# 拆包
x_, y_ = v1
print("拆包结果: ", x_, " ", y_)

# repr()函数调用
print("repr(v1): ", repr(v1))

# == 判断相等性
v2 = Vector2d(3, 3)
v3 = Vector2d(3, 4)
print("{} == {}: ".format(v1, v2), v1 == v2)
print("{} == {}: ".format(v1, v3), v1 == v3)

# print()函数调用
print("print(v1): ", end=" ")
print(v1)

# bytes()函数调用
print("bytes(v1)", bytes(v1))

# abs()函数调用
print("abs(v1): ", abs(v1))

# bool()函数调用
v0 = Vector2d(0, 0)
print("bool(v1): ", bool(v1))
print("bool(v0): ", bool(v0))

# 从字节序列重建
v1_clone = Vector2d.frombytes(bytes(v1))
print("从v1字节序列重建结果:", v1_clone)

# 格式化输出
print("以.2f格式化输出v1: ", format(v1, ".2f"))
print("以极坐标形式输出v1: ", format(v1, ".2fp"))

# 属性仅读
try:
    v1.x = 4.0
except:
    print("尝试修改v1的属性x：AttributeError")

# hash
print("v1 hash值：", hash(v1))

# 存入set
print(set([v1, v2]))

v1向量 x分量:  3.0
v1向量 y分量:  4.0
拆包结果:  3.0   4.0
repr(v1):  Vector2d(3.0, 4.0)
(3.0, 4.0) == (3.0, 3.0):  False
(3.0, 4.0) == (3.0, 4.0):  True
print(v1):  (3.0, 4.0)
bytes(v1) b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
abs(v1):  5.0
bool(v1):  True
bool(v0):  False
从v1字节序列重建结果: (3.0, 4.0)
以.2f格式化输出v1:  (3.00, 4.00)
以极坐标形式输出v1:  <5.00, 0.93>
尝试修改v1的属性x：AttributeError
v1 hash值： 7
{Vector2d(3.0, 3.0), Vector2d(3.0, 4.0)}


## 私有属性以及“受保护的”属性

上述实现的自定义向量类实际上利用了Python的一个语言特性 —— 名称改写(name mangling)

使用两个前导下划线的属性在存入实例的\_\_dict\_\_中前，Python会首先对这个属性名进行改写：\_\_param -> \_cls\_\_param。

显然，改写后，用户不会无意覆盖\_\_param的属性值；但是这种方式无法避免有意赋值（直接对\_cls\_\_param赋值）

上述功能是Python解释器会执行的操作，但是也可以主动使用命名约定决定哪些属性是可被访问的，哪些属性应当被视为“私有属性”。例如约定有一个前导下划线的属性为“私有属性”。值得注意的是，此时Python并不会对该变量执行特殊操作，这一规则仅是一种命名约定，而不具有强制性。

## 其他需求以及优化方法

### \_\_slots\_\_

若在程序运行过程中不需要动态增加对象的属性，则可以使用\_\_slots\_\_替换\_\_dict\_\_，实现以元组的形式存储属性，这样能够节省大量空间。

使用\_\_slots\_\_后实例不可再增加属性。但是若将\_\_dict\_\_计入\_\_slots\_\_，则实例在\_\_slots\_\_中保存属性，并且支持动态添加属性至\_\_dict\_\_。这样的操作显然得不偿失。

此外，子类不会主动继承父类的\_\_slots\_\_属性 —— 若有必要，需要在子类中重新定义\_\_slots\_\_

若要支持弱引用，则需要将'\_\_weakref\_\_'添加至\_\_slots\_\_

### 覆盖属性值

上述定义的向量类有两种属性，其一是typecode属性，这一属性没有定义在\_\_init\_\_中；其二是属性x和属性y，这两个属性定义在\_\_init\_\_中。

当使用self.typecode调用typecode属性时，由于实例本身没有这一属性，因此获取到的实际上是Vector2d.typecode。

若使用Vector2d.typecode覆盖typecode的属性值，则会改变该类所有实例的typecode属性值；对于属性x和属性y则不会有这一效果。

可以通过创建子类来更有针对性的实现上述功能，具体来说，在子类中改变类属性的值
```Python
class ShortVector2d(Vector2d):
    typecode = "f"
```

In [18]:
class ShortVector2d(Vector2d):
    typecode = "f"

v1 = Vector2d(1, 2)
v2 = Vector2d(1, 2)

# 覆盖v1typecode
v1.typecode = "f"
print("v1的typecode属性覆盖后v2的typecode：", v2.typecode)
Vector2d.typecode = "f"
v3 = Vector2d(1, 2)
print("Vector2d的typecode属性覆盖后v2的typecode：", v2.typecode)

v4 = ShortVector2d(1, 2)
print("v4的typecode属性：", v4.typecode)

v1的typecode属性覆盖后v2的typecode： d
Vector2d的typecode属性覆盖后v2的typecode： f
v4的typecode属性： f


## 总结
* 鸭子类型是在Python中实现符合Python风格的自定义类的不二法宝，通过对指定的方法进行实现，可以让自定义类表现得像一个内置类型。
* Python中有多个方法用于获取以及生成类的字符串和字节序列表示形式，这些方法分别被不同的内置函数调用以实现各类输出需求。
* 通过实现\_\_format\_\_方法，可以使得自定义类支持格式化输出，并且可以自行定义格式说明的解释方法以支持自定义的格式化输出模式
* 通过实现\_\_hash\_\_以及\_\_eq\_\_方法可以实现自定义类实例的可散列化。并且根据文档推荐，应当使用实例各属性的hash值来构建该实例的hash值。
* Python支持使用双前导下划线命名属性以实现名称改写，避免该对象的无意修改。但是这一特性仅能够避免意外修改而不能防止有意赋值。此外，还可以使用自行设定的命名约定来避免意外覆盖属性值，这一方法仅是一种约定而不具有强制性。
* 使用\_\_slots\_\_属性替换\_\_dict\_\_存储类属性能够有效节约内存，但是其具有一些本质上的特性需要特别处理以实现预期功能，规避潜在漏洞。
* Alex Martelli：不要检查它是不是鸭子：检查它的叫声像不像鸭子，它的走路姿势像不像鸭子，等等。具体检查什么取决于你想使用语言的哪些行为。